In [12]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import numpy as np

In [13]:
# Fungsi untuk menormalkan nama provinsi
def normalize_provinsi_names(df, column):
    df[column] = df[column].str.strip().str.title()
    return df

In [14]:
# Load datasets
statistik_covid_path = "Statistik_Harian_per_Provinsi_COVID19_Indonesia_Rev.csv"
kepadatan_penduduk_path = "Kepadatan Penduduk menurut Provinsi.xlsx"
jumlah_penduduk_path = "Jumlah Penduduk.xlsx"

In [15]:
# Membaca dataset
statistik_covid = pd.read_csv(statistik_covid_path)
kepadatan_penduduk = pd.read_excel(kepadatan_penduduk_path)
jumlah_penduduk = pd.read_excel(jumlah_penduduk_path)

In [16]:
# Seleksi kolom penting
statistik_covid_cleaned = statistik_covid[
    ["Provinsi", "Tanggal", "Kasus_Terkonfirmasi_Akumulatif", 
     "Penambahan_Harian_Kasus_Terkonf", "Kasus_Sembuh_Akumulatif", 
     "Kasus_Meninggal_Akumulatif"]
].dropna()

In [17]:
kepadatan_penduduk_cleaned = kepadatan_penduduk[
    ["Provinsi", "Kepadatan Penduduk menurut Provinsi (jiwa/km2)"]
].dropna()

In [18]:
jumlah_penduduk_cleaned = jumlah_penduduk[
    ["Provinsi", "Jumlah Penduduk Hasil Proyeksi Menurut Provinsi dan Jenis Kelamin (Ribu Jiwa)"]
].dropna()

In [19]:
# Rename kolom untuk konsistensi
jumlah_penduduk_cleaned.rename(columns={
    "Jumlah Penduduk Hasil Proyeksi Menurut Provinsi dan Jenis Kelamin (Ribu Jiwa)": "Jumlah_Penduduk"
}, inplace=True)

In [20]:
# Normalisasi nama provinsi
statistik_covid_cleaned = normalize_provinsi_names(statistik_covid_cleaned, "Provinsi")
kepadatan_penduduk_cleaned = normalize_provinsi_names(kepadatan_penduduk_cleaned, "Provinsi")
jumlah_penduduk_cleaned = normalize_provinsi_names(jumlah_penduduk_cleaned, "Provinsi")

In [26]:
# Gabungkan dataset berdasarkan "Provinsi"
merged_data = statistik_covid_cleaned.merge(
    kepadatan_penduduk_cleaned, on="Provinsi", how="inner"
).merge(
    jumlah_penduduk_cleaned, on="Provinsi", how="inner"
)

In [27]:
print(merged_data[['Kasus_Terkonfirmasi_Akumulatif', 'Kasus_Meninggal_Akumulatif', 'Kasus_Sembuh_Akumulatif', 'Jumlah_Penduduk']].dtypes)


Kasus_Terkonfirmasi_Akumulatif    float64
Kasus_Meninggal_Akumulatif        float64
Kasus_Sembuh_Akumulatif           float64
Jumlah_Penduduk                    object
dtype: object


In [28]:
# Pastikan kolom yang terlibat dalam perhitungan memiliki tipe data numerik
merged_data['Kasus_Terkonfirmasi_Akumulatif'] = pd.to_numeric(merged_data['Kasus_Terkonfirmasi_Akumulatif'], errors='coerce')
merged_data['Kasus_Meninggal_Akumulatif'] = pd.to_numeric(merged_data['Kasus_Meninggal_Akumulatif'], errors='coerce')
merged_data['Kasus_Sembuh_Akumulatif'] = pd.to_numeric(merged_data['Kasus_Sembuh_Akumulatif'], errors='coerce')
merged_data['Jumlah_Penduduk'] = pd.to_numeric(merged_data['Jumlah_Penduduk'], errors='coerce')

In [29]:
# Cek apakah ada NaN setelah konversi
print(merged_data[['Kasus_Terkonfirmasi_Akumulatif', 'Kasus_Meninggal_Akumulatif', 
                   'Kasus_Sembuh_Akumulatif', 'Jumlah_Penduduk']].isna().sum())

Kasus_Terkonfirmasi_Akumulatif       0
Kasus_Meninggal_Akumulatif           0
Kasus_Sembuh_Akumulatif              0
Jumlah_Penduduk                   7875
dtype: int64


In [30]:
# Menghapus baris yang memiliki NaN jika diperlukan
merged_data = merged_data.dropna(subset=['Kasus_Terkonfirmasi_Akumulatif', 'Kasus_Meninggal_Akumulatif',
                                         'Kasus_Sembuh_Akumulatif', 'Jumlah_Penduduk'])

In [31]:
# Tambahkan fitur tambahan
merged_data['Rasio_Kasus_Penduduk'] = merged_data['Kasus_Terkonfirmasi_Akumulatif'] / merged_data['Jumlah_Penduduk']
merged_data['Rasio_Meninggal_Penduduk'] = merged_data['Kasus_Meninggal_Akumulatif'] / merged_data['Jumlah_Penduduk']
merged_data['Rasio_Sembuh_Penduduk'] = merged_data['Kasus_Sembuh_Akumulatif'] / merged_data['Jumlah_Penduduk']

In [34]:
# Seleksi provinsi untuk Jawa Timur dan DKI Jakarta
filtered_data = merged_data[merged_data['Provinsi'].isin(["Jawa Timur", "Dki Jakarta"])]

In [35]:
# Cek apakah filtered_data memiliki data setelah filter
print(filtered_data.shape)
print(filtered_data.head())


(0, 11)
Empty DataFrame
Columns: [Provinsi, Tanggal, Kasus_Terkonfirmasi_Akumulatif, Penambahan_Harian_Kasus_Terkonf, Kasus_Sembuh_Akumulatif, Kasus_Meninggal_Akumulatif, Kepadatan Penduduk menurut Provinsi (jiwa/km2), Jumlah_Penduduk, Rasio_Kasus_Penduduk, Rasio_Meninggal_Penduduk, Rasio_Sembuh_Penduduk]
Index: []


In [36]:
# Cek kolom yang ada setelah filter
print(filtered_data.columns)


Index(['Provinsi', 'Tanggal', 'Kasus_Terkonfirmasi_Akumulatif',
       'Penambahan_Harian_Kasus_Terkonf', 'Kasus_Sembuh_Akumulatif',
       'Kasus_Meninggal_Akumulatif',
       'Kepadatan Penduduk menurut Provinsi (jiwa/km2)', 'Jumlah_Penduduk',
       'Rasio_Kasus_Penduduk', 'Rasio_Meninggal_Penduduk',
       'Rasio_Sembuh_Penduduk'],
      dtype='object')


In [37]:
# Menampilkan semua provinsi yang tersedia dalam dataset
print(merged_data['Provinsi'].unique())


[]


In [38]:
if filtered_data.empty:
    print("Data untuk provinsi yang dipilih tidak ditemukan.")
else:
    # Lanjutkan proses clustering
    scaled_features = scaler.fit_transform(filtered_data[features])


Data untuk provinsi yang dipilih tidak ditemukan.


In [39]:
# Cek provinsi yang ada di setiap dataset sebelum merge
print(statistik_covid['Provinsi'].unique())
print(kepadatan_penduduk['Provinsi'].unique())
print(jumlah_penduduk['Provinsi'].unique())


['Aceh' 'Bali' 'Banten' 'Kepulauan Bangka Belitung' 'Bengkulu'
 'Daerah Istimewa Yogyakarta' 'DKI Jakarta' 'Jambi' 'Jawa Barat'
 'Jawa Tengah' 'Jawa Timur' 'Kalimantan Barat' 'Kalimantan Timur'
 'Kalimantan Tengah' 'Kalimantan Selatan' 'Kalimantan Utara'
 'Kepulauan Riau' 'Nusa Tenggara Barat' 'Sumatera Selatan'
 'Sumatera Barat' 'Sulawesi Utara' 'Sumatera Utara' 'Sulawesi Tenggara'
 'Sulawesi Selatan' 'Sulawesi Tengah' 'Lampung' 'Riau' 'Maluku Utara'
 'Maluku' 'Papua Barat' 'Papua' 'Sulawesi Barat' 'Nusa Tenggara Timur'
 'Gorontalo' 'Indonesia' nan]
[nan 'Aceh' 'Bali' 'Banten' 'Bengkulu' 'Daerah Istimewa Yogyakarta'
 'DKI Jakarta' 'Gorontalo' 'INDONESIA' 'Jambi' 'Jawa Barat' 'Jawa Tengah'
 'Jawa Timur' 'Kalimantan Barat' 'Kalimantan Selatan' 'Kalimantan Tengah'
 'Kalimantan Timur' 'Kalimantan Utara' 'Kepulauan Bangka Belitung'
 'Kepulauan Riau' 'Lampung' 'Maluku' 'Maluku Utara' 'Nusa Tenggara Barat'
 'Nusa Tenggara Timur' 'Papua' 'Papua Barat' 'Riau' 'Sulawesi Barat'
 'Sulawesi Selata

In [40]:
# Cek nama provinsi setelah normalisasi
print(statistik_covid_cleaned['Provinsi'].unique())
print(kepadatan_penduduk_cleaned['Provinsi'].unique())
print(jumlah_penduduk_cleaned['Provinsi'].unique())


['Aceh' 'Bali' 'Banten' 'Kepulauan Bangka Belitung' 'Bengkulu'
 'Daerah Istimewa Yogyakarta' 'Dki Jakarta' 'Jambi' 'Jawa Barat'
 'Jawa Tengah' 'Jawa Timur' 'Kalimantan Barat' 'Kalimantan Timur'
 'Kalimantan Tengah' 'Kalimantan Selatan' 'Kalimantan Utara'
 'Kepulauan Riau' 'Nusa Tenggara Barat' 'Sumatera Selatan'
 'Sumatera Barat' 'Sulawesi Utara' 'Sumatera Utara' 'Sulawesi Tenggara'
 'Sulawesi Selatan' 'Sulawesi Tengah' 'Lampung' 'Riau' 'Maluku Utara'
 'Maluku' 'Papua Barat' 'Papua' 'Sulawesi Barat' 'Nusa Tenggara Timur'
 'Gorontalo' 'Indonesia']
['Aceh' 'Bali' 'Banten' 'Bengkulu' 'Daerah Istimewa Yogyakarta'
 'Dki Jakarta' 'Gorontalo' 'Indonesia' 'Jambi' 'Jawa Barat' 'Jawa Tengah'
 'Jawa Timur' 'Kalimantan Barat' 'Kalimantan Selatan' 'Kalimantan Tengah'
 'Kalimantan Timur' 'Kalimantan Utara' 'Kepulauan Bangka Belitung'
 'Kepulauan Riau' 'Lampung' 'Maluku' 'Maluku Utara' 'Nusa Tenggara Barat'
 'Nusa Tenggara Timur' 'Papua' 'Papua Barat' 'Riau' 'Sulawesi Barat'
 'Sulawesi Selatan' 'Sula

In [41]:
# Cek hasil merge
print(merged_data['Provinsi'].unique())


[]


In [42]:
if merged_data.empty:
    print("Data setelah merge kosong, periksa kesesuaian nama provinsi atau proses penggabungan.")
else:
    # Lanjutkan analisis
    filtered_data = merged_data[merged_data['Provinsi'].isin(["Jawa Timur", "Dki Jakarta"])]
    print(filtered_data.head())


Data setelah merge kosong, periksa kesesuaian nama provinsi atau proses penggabungan.


In [43]:
# Normalisasi nama provinsi menjadi format yang konsisten
statistik_covid_cleaned['Provinsi'] = statistik_covid_cleaned['Provinsi'].str.strip().str.title()
kepadatan_penduduk_cleaned['Provinsi'] = kepadatan_penduduk_cleaned['Provinsi'].str.strip().str.title()
jumlah_penduduk_cleaned['Provinsi'] = jumlah_penduduk_cleaned['Provinsi'].str.strip().str.title()


In [44]:
statistik_covid_cleaned = statistik_covid_cleaned[statistik_covid_cleaned['Provinsi'] != 'Indonesia']
kepadatan_penduduk_cleaned = kepadatan_penduduk_cleaned[kepadatan_penduduk_cleaned['Provinsi'] != 'Indonesia']
jumlah_penduduk_cleaned = jumlah_penduduk_cleaned[jumlah_penduduk_cleaned['Provinsi'] != 'Indonesia']

statistik_covid_cleaned = statistik_covid_cleaned.dropna(subset=['Provinsi'])
kepadatan_penduduk_cleaned = kepadatan_penduduk_cleaned.dropna(subset=['Provinsi'])
jumlah_penduduk_cleaned = jumlah_penduduk_cleaned.dropna(subset=['Provinsi'])


In [45]:
merged_data = pd.merge(statistik_covid_cleaned, kepadatan_penduduk_cleaned, on='Provinsi', how='inner')
merged_data = pd.merge(merged_data, jumlah_penduduk_cleaned, on='Provinsi', how='inner')


In [47]:
import pandas as pd

# Normalisasi nama provinsi untuk memastikan format yang konsisten
def normalize_provinsi(df):
    df['Provinsi'] = df['Provinsi'].str.strip().str.title()  # Strip whitespace dan ubah ke format title
    return df

# Memuat dataset
statistik_covid = pd.read_csv("Statistik_Harian_per_Provinsi_COVID19_Indonesia_Rev.csv")
kepadatan_penduduk = pd.read_csv("Kepadatan Penduduk menurut Provinsi.xlsx")
jumlah_penduduk = pd.read_csv("Jumlah Penduduk.xlsx")

# Normalisasi nama provinsi di setiap dataset
statistik_covid_cleaned = normalize_provinsi(statistik_covid)
kepadatan_penduduk_cleaned = normalize_provinsi(kepadatan_penduduk)
jumlah_penduduk_cleaned = normalize_provinsi(jumlah_penduduk)

# Menghapus baris dengan nilai 'Indonesia' atau NaN di kolom 'Provinsi'
statistik_covid_cleaned = statistik_covid_cleaned[statistik_covid_cleaned['Provinsi'] != 'Indonesia']
kepadatan_penduduk_cleaned = kepadatan_penduduk_cleaned[kepadatan_penduduk_cleaned['Provinsi'] != 'Indonesia']
jumlah_penduduk_cleaned = jumlah_penduduk_cleaned[jumlah_penduduk_cleaned['Provinsi'] != 'Indonesia']

statistik_covid_cleaned = statistik_covid_cleaned.dropna(subset=['Provinsi'])
kepadatan_penduduk_cleaned = kepadatan_penduduk_cleaned.dropna(subset=['Provinsi'])
jumlah_penduduk_cleaned = jumlah_penduduk_cleaned.dropna(subset=['Provinsi'])

# Periksa nama provinsi setelah normalisasi
print("Provinsi di statistik_covid_cleaned:")
print(statistik_covid_cleaned['Provinsi'].unique())

print("Provinsi di kepadatan_penduduk_cleaned:")
print(kepadatan_penduduk_cleaned['Provinsi'].unique())

print("Provinsi di jumlah_penduduk_cleaned:")
print(jumlah_penduduk_cleaned['Provinsi'].unique())

# Merge data
merged_data = pd.merge(statistik_covid_cleaned, kepadatan_penduduk_cleaned, on='Provinsi', how='inner')
merged_data = pd.merge(merged_data, jumlah_penduduk_cleaned, on='Provinsi', how='inner')

# Cek hasil merge
if merged_data.empty:
    print("Data setelah merge kosong, periksa kesesuaian nama provinsi atau proses penggabungan.")
else:
    # Lanjutkan analisis
    filtered_data = merged_data[merged_data['Provinsi'].isin(["Jawa Timur", "Dki Jakarta"])]
    print(filtered_data.head())


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xa4 in position 14: invalid start byte